In [1]:
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [2]:
load_dotenv()
key = os.getenv("QWEN_API_KEY")
if not key:
    raise ValueError("Put QWEN_API_KEY=sk-xxxxxxxxxx into .env file")
llm = ChatOpenAI(
    model="qwen-max",
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",
    temperature=0.0,
    api_key=key
)

In [6]:
data_file_path = "../fl_data_bot/data/freelancer_earnings_bd.csv"
df = pd.read_csv(data_file_path)
#df.describe()
df.head()

,Freelancer_ID,Job_Category,Platform,Experience_Level,Client_Region,Payment_Method,Job_Completed,Earnings_USD,Hourly_Rate,Job_Success_Rate,Client_Rating,Job_Duration_Days,Project_Type,Rehire_Rate,Marketing_Spend
0,1,Web Development,Fiverr,Beginner,Asia,Mobile Banking,180,1620,95.79,68.73,3.18,1,Fixed,40.19,53
1,2,App Development,Fiverr,Beginner,Australia,Mobile Banking,218,9078,86.38,97.54,3.44,54,Fixed,36.53,486
2,3,Web Development,Fiverr,Beginner,UK,Crypto,27,3455,85.17,86.60,4.20,46,Hourly,74.05,489
3,4,Data Entry,PeoplePerHour,Intermediate,Asia,Bank Transfer,17,5577,14.37,79.93,4.47,41,Hourly,27.58,67
4,5,Digital Marketing,Upwork,Expert,Asia,Crypto,245,5898,99.37,57.80,5.00,41,Hourly,69.09,489


In [11]:
agent = create_pandas_dataframe_agent(
    llm,
    df,
    agent_type="tool-calling",  # Use the modern "tool-calling" agent type
    allow_dangerous_code=True,  # Allow execution of Python code (use with caution)
    verbose=True,               # Enable verbose logging for debugging
)

In [12]:
response = agent.invoke("What are the columns in the dataset and their data types?")
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.dtypes.to_dict()'}`


{'Freelancer_ID': dtype('int64'), 'Job_Category': dtype('O'), 'Platform': dtype('O'), 'Experience_Level': dtype('O'), 'Client_Region': dtype('O'), 'Payment_Method': dtype('O'), 'Job_Completed': dtype('int64'), 'Earnings_USD': dtype('int64'), 'Hourly_Rate': dtype('float64'), 'Job_Success_Rate': dtype('float64'), 'Client_Rating': dtype('float64'), 'Job_Duration_Days': dtype('int64'), 'Project_Type': dtype('O'), 'Rehire_Rate': dtype('float64'), 'Marketing_Spend': dtype('int64')}The dataset contains the following columns with their respective data types:

- `Freelancer_ID`: integer
- `Job_Category`: object (string)
- `Platform`: object (string)
- `Experience_Level`: object (string)
- `Client_Region`: object (string)
- `Payment_Method`: object (string)
- `Job_Completed`: integer
- `Earnings_USD`: integer
- `Hourly_Rate`: float
- `Job_Success_Rate`: float
- `Client_Rating`: float
- 

In [14]:
response = agent.invoke("how much more profitable freelancers taking crypto?")
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df_crypto = df[df['Payment_Method'] == 'Crypto']\navg_earnings_crypto = df_crypto['Earnings_USD'].mean()\n\ndf_other = df[df['Payment_Method'] != 'Crypto']\navg_earnings_other = df_other['Earnings_USD'].mean()\n\navg_earnings_crypto, avg_earnings_other, avg_earnings_crypto - avg_earnings_other"}`
responded: To evaluate how much more profitable freelancers taking crypto are compared to those using other payment methods, we can calculate the average earnings of freelancers who use crypto and compare it to the average earnings of those who use other methods. 

Let's proceed with this analysis.

(np.float64(5139.301556420233), np.float64(4973.993036211699), np.float64(165.308520208534))The average earnings for freelancers using Crypto as a payment method is approximately $5139.30, while the average earnings for freelancers using other payment methods is approximately $4974. 

The difference in average ear

In [15]:
response = agent.invoke("what is income distribution per region?")
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.groupby('Client_Region')['Earnings_USD'].sum()"}`


Client_Region
Asia           1453412
Australia      1479897
Canada         1316133
Europe         1281319
Middle East    1417408
UK             1413185
USA            1422901
Name: Earnings_USD, dtype: int64The income distribution per region based on the earnings in USD is as follows:

- Asia: $1,453,412
- Australia: $1,479,897
- Canada: $1,316,133
- Europe: $1,281,319
- Middle East: $1,417,408
- UK: $1,413,185
- USA: $1,422,901

These figures represent the total earnings of freelancers from each respective region.

> Finished chain.
The income distribution per region based on the earnings in USD is as follows:

- Asia: $1,453,412
- Australia: $1,479,897
- Canada: $1,316,133
- Europe: $1,281,319
- Middle East: $1,417,408
- UK: $1,413,185
- USA: $1,422,901

These figures represent the total earnings of freelancers from each respective region.


In [16]:
response = agent.invoke("percentage of expert level freelancers completed less than 100 project")
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['Experience_Level'] == 'Expert') & (df['Job_Completed'] < 100)].shape[0] / df[df['Experience_Level'] == 'Expert'].shape[0] * 100"}`


33.85335413416536533.85% of expert level freelancers have completed less than 100 projects.

> Finished chain.
33.85% of expert level freelancers have completed less than 100 projects.


In [17]:
response = agent.invoke("Насколько выше доход у фрилансеров, принимающих оплату в криптовалюте, по сравнению с другими способами оплаты?")
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Payment_Method'].unique().tolist()"}`


['Mobile Banking', 'Crypto', 'Bank Transfer', 'PayPal']
Invoking: `python_repl_ast` with `{'query': "crypto_earnings = df[df['Payment_Method'] == 'Crypto']['Earnings_USD'].mean()"}`



Invoking: `python_repl_ast` with `{'query': "other_methods_earnings = df[df['Payment_Method'] != 'Crypto']['Earnings_USD'].mean()"}`



Invoking: `python_repl_ast` with `{'query': 'percentage_increase = ((crypto_earnings - other_methods_earnings) / other_methods_earnings) * 100; percentage_increase'}`


3.3234570093897147Доход фрилансеров, принимающих оплату в криптовалюте, в среднем на 3.32% выше, чем тех, кто использует другие методы оплаты.

> Finished chain.
Доход фрилансеров, принимающих оплату в криптовалюте, в среднем на 3.32% выше, чем тех, кто использует другие методы оплаты.


In [18]:
response = agent.invoke("Как распределяется доход фрилансеров в зависимости от региона проживания?")
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.groupby('Client_Region')['Earnings_USD'].mean().sort_values(ascending=False)"}`
responded: Чтобы ответить на этот вопрос, нам нужно сгруппировать данные по региону клиентов (Client_Region) и рассчитать средний доход (Earnings_USD). Мы можем использовать функцию groupby в pandas для этого. Давайте выполним этот анализ.

Client_Region
Canada         5350.134146
Asia           5172.284698
UK             5047.089286
Australia      4966.097315
Europe         4890.530534
USA            4872.948630
Middle East    4870.817869
Name: Earnings_USD, dtype: float64Вот средний доход фрилансеров в зависимости от региона проживания клиентов (в USD):

1. Канада: 5350.13
2. Азия: 5172.28  
3. Великобритания: 5047.09
4. Австралия: 4966.10
5. Европа: 4890.53
6. США: 4872.95
7. Ближний Восток: 4870.82

Можно заметить, что наибольший средний доход у фрилансеров, работающих с клиентами из Канады, а наименьший - с клиента

In [19]:
response = agent.invoke("Какой процент фрилансеров, считающих себя экспертами, выполнил менее 100 проектов?")
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['Experience_Level'] == 'Expert') & (df['Job_Completed'] < 100)].shape[0] / df[df['Experience_Level'] == 'Expert'].shape[0] * 100"}`


33.853354134165365Приблизительно 33.85% фрилансеров, считающих себя экспертами, выполнили менее 100 проектов.

> Finished chain.
Приблизительно 33.85% фрилансеров, считающих себя экспертами, выполнили менее 100 проектов.
